# Demostraciones prácticas de la entrega 2
Este archivo está pensado para ser visualizado en un navegador web utilizando Voilà. Para ejecutarlo, se debe
ejecutar la siguiente línea en una terminal:
```bash
voila anomaly-detection.ipynb
```

En caso de no tener instalado alguno de los paquetes necesarios, se puede descomentar la sección inferior.

*Juan Francisco Mier Montoto, UO283319*

In [2]:
# Para ejecutar los ejemplos, se necesitan las siguientes librerías:

# %pip install pandas
# %pip install matplotlib
# %pip install numpy
# %pip install scikit-learn
# %pip install ipywidgets
# %pip install voila

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors, LocalOutlierFactor
from sklearn.datasets import make_blobs
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

****

In [4]:
media, cov, n = np.array([0.01, 0.74]), np.array([[1.0, 0.5], [0.5, 1.0]]), 1000
rng = np.random.default_rng(7401) # generador de números aleatorios
data = rng.multivariate_normal(media, cov, n) # generación de muestras
x, y = data.T # separación de las muestras en dos vectores

## Ejemplo 1
Demostración del funcionamiento de la distancia de Mahalanobis con distribuciones multivariantes.

In [5]:
def plot_mahalanobis(umbral):
	cov_inv = np.linalg.inv(cov) # inversa de la covarianza para calcular la distancia de Mahalanobis

	dist = [distance.mahalanobis(media, [x[i], y[i]], cov_inv) for i in range(n)]

	# dibujar el valor absoluto para las distancias y el umbral
	dist_abs = np.abs(dist)
	plt.title('Distancia absoluta de Mahalanobis')
	plt.plot(dist_abs, 'x')
	plt.axhline(y=umbral, color='r', linestyle='--', label='Umbral')
	plt.scatter(np.nonzero(dist_abs > umbral)[0], dist_abs[dist_abs > umbral], color='red', label='Anomalía')
	plt.legend()
	plt.show()

	# dibuajar el plot de las muestras originales coloreando las que están por encima del umbral
	plt.title('Muestras coloreadas por encima del umbral')
	# colorear las muestras "normales" en una escala según su distancia de Mahalanobis
	plt.scatter(x[dist_abs < umbral], y[dist_abs < umbral], c=dist_abs[dist_abs < umbral], cmap='viridis', label='Muestra normal')
	plt.colorbar()
	plt.scatter(x[dist_abs > umbral], y[dist_abs > umbral], color='red', label='Anomalía')
	plt.legend()
	plt.show()

interact(plot_mahalanobis, umbral=(0, 5, 0.1))

interactive(children=(FloatSlider(value=2.0, description='umbral', max=5.0), Output()), _dom_classes=('widget-…

<function __main__.plot_mahalanobis(umbral)>

## Ejemplo 2
Detección de anomalías basada en proximidad (knn)

In [6]:
def knn(k, threshold):
	# se calcula la distancia media de los k vecinos más cercanos
	neighbours = NearestNeighbors(n_neighbors=k).fit(data)
	dist, _ = neighbours.kneighbors(data)
	dist = np.mean(dist, axis=1)

	# se dibuja la distancia media de los k vecinos más cercanos y el umbral
	plt.plot(dist, 'x', color='purple')
	plt.title('Distancia media de los ' + str(k) + ' vecinos más cercanos')
	plt.axhline(y=threshold, color='r', linestyle='--', label='Umbral')
	plt.scatter(np.nonzero(dist > threshold)[0], dist[dist > threshold], color='orange', label='Anomalía')
	plt.legend()
	plt.show()

	# representación de las muestras coloreando las que están por encima del umbral
	plt.title('Muestras coloreadas por encima del umbral')
	plt.plot(x, y, 'x', color='green', label='Muestra normal')
	plt.plot(x[dist > threshold], y[dist > threshold], 'x', color='red', label='Muestra anómala')

	# se dibuja el radio de la circunferencia de los k vecinos más cercanos de las muestras anómalas
	for i in np.nonzero(dist > threshold)[0]:
		circ = plt.Circle((x[i], y[i]), dist[i], color='red', fill=False)
		plt.gcf().gca().add_artist(circ)
		circ = plt.Circle((x[i], y[i]), threshold, color='black', fill=False, linestyle='--')
		plt.gcf().gca().add_artist(circ)
		# no sé cómo añadir la etiqueta de la circunferencia a la leyenda :(
	plt.plot(media[0], media[1], 'o', color='black', label='Media')
	plt.legend()
	plt.show()

interact(knn, k=(5, 10, 1), threshold=(0.1, 1, 0.1))

interactive(children=(IntSlider(value=7, description='k', max=10, min=5), FloatSlider(value=0.5, description='…

<function __main__.knn(k, threshold)>

## Ejemplo 3
Clasificación de anomalías con KNN y LOF

In [7]:
def knn_class(k, threshold):
	# se calcula la distancia media de los k vecinos más cercanos
	neighbours = NearestNeighbors(n_neighbors=k).fit(data)
	dist, _ = neighbours.kneighbors(data)
	dist = np.mean(dist, axis=1)

	# se clasifican las muestras en normales y anómalas según el umbral
	labels = np.zeros(n)
	labels[dist > threshold] = 1

	# se crea un clasificador KNN para clasificar las muestras
	clf = KNeighborsClassifier(n_neighbors=k)
	clf.fit(data, labels)

	# se crea una malla para clasificar todas las muestras del plano
	x_min, x_max = x.min() - 1, x.max() + 1
	y_min, y_max = y.min() - 1, y.max() + 1
	xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
	Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
	Z = Z.reshape(xx.shape)

	# se dibuja la malla y las muestras
	plt.contourf(xx, yy, Z, alpha=0.4)
	plt.scatter(x, y, c=labels, cmap='viridis')
	plt.title('Clasificación (KNN) de las muestras')
	plt.show()

interact(knn_class, k=(5, 10, 1), threshold=(0.1, 0.3, 0.1))

interactive(children=(IntSlider(value=7, description='k', max=10, min=5), FloatSlider(value=0.2, description='…

<function __main__.knn_class(k, threshold)>

In [8]:
def lof_class(k):
	# se crea un clasificador SVM para clasificar las muestras
	clf = LocalOutlierFactor(n_neighbors=k, novelty=True)
	clf.fit(data)

	# se crea una malla para clasificar todas las muestras del plano
	x_min, x_max = x.min() - 1, x.max() + 1
	y_min, y_max = y.min() - 1, y.max() + 1
	xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
	Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
	Z = Z.reshape(xx.shape)

	# se dibuja la malla y las muestras
	plt.contourf(xx, yy, Z, alpha=0.4)
	plt.scatter(x, y, c=clf.predict(data), cmap='viridis')
	plt.title('Clasificación (LOF) de las muestras')
	plt.show()

interact(lof_class, k=(5, 25, 1))

interactive(children=(IntSlider(value=15, description='k', max=25, min=5), Output()), _dom_classes=('widget-in…

<function __main__.lof_class(k)>